In [1]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import sklearn
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
import nltk
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.metrics import pairwise_distances
import tensorflow_hub as hub
import tensorflow as tf
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

In [2]:
### Description for train data
desc_files = len(os.listdir('../descriptions_train'))
all_desc_train = []

for i in range(desc_files):
    empty_str = ''
    for line in open(f'../descriptions_train/{i}.txt'):
        empty_str += line.replace('\n',' ')
    all_desc_train.append(empty_str)

In [3]:
### Tags for train data
tag_files = len(os.listdir('../tags_train'))
all_tags_train = []

for i in range(tag_files):
    nouns = ''
    for line in open(f'../tags_train/{i}.txt'):
        nouns += line.replace(':',' ')
    all_tags_train.append(nouns.replace('\n', ' '))

In [4]:
### Description for test data
desc_files = len(os.listdir('../descriptions_test'))
all_desc_test = []

for i in range(desc_files):
    empty_str = ''
    for line in open(f'../descriptions_test/{i}.txt'):
        empty_str += line.replace('\n',' ')
    all_desc_test.append(empty_str)

In [5]:
### Tags for test data
tag_files = len(os.listdir('../tags_test'))
all_tags_test = []

for i in range(tag_files):
    nouns = ''
    for line in open(f'../tags_test/{i}.txt'):
        nouns += line.replace(':',' ')
    all_tags_test.append(nouns.replace('\n', ' '))

In [6]:
all_docs = []
all_docs.extend(all_desc_train)
all_docs.extend(all_desc_test)
all_docs.extend(all_tags_train)
all_docs.extend(all_tags_test)

In [7]:
train_1000 = pd.read_csv('../features_train/features_resnet1000_train.csv', header=None)
train_2048 = pd.read_csv('../features_train/features_resnet1000intermediate_train.csv', header=None)
test_1000 = pd.read_csv('../features_test/features_resnet1000_test.csv', header=None)
test_2048 = pd.read_csv('../features_test/features_resnet1000intermediate_test.csv', header=None)

In [8]:
def get_num(string):
    string = string.replace('.', ' ').replace('/', ' ')
    num = [int(s) for s in string.split() if s.isdigit()]
    return num[0]

def parse_to_numpy(pd):
    images_idx = []
    for string in pd[0]:
        images_idx.append(get_num(string))

    pd.insert(1, "Image_Index", images_idx, True)
    pd = pd.sort_values(by=['Image_Index'])
    pd = pd.reset_index(drop=True)
    del pd['Image_Index']
    del pd[0]
    np = pd.to_numpy()
    return np

In [9]:
train_1000 = parse_to_numpy(train_1000)
train_2048 = parse_to_numpy(train_2048)
test_1000 = parse_to_numpy(test_1000)
test_2048 = parse_to_numpy(test_2048)

In [10]:
### Google embedding
#tf.enable_eager_execution()
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
### Overwrite old train_desc and test_desc
train_desc = embed(all_desc_train).numpy()
test_desc = embed(all_desc_test).numpy()
train_tags = embed(all_tags_train).numpy()
test_tags = embed(all_tags_test).numpy()

In [11]:
train_pic = np.hstack((train_1000, train_2048))#, train_tags))
test_pic = np.hstack((test_1000, test_2048))#, test_tags))

In [12]:
# word2vec
# path = get_tmpfile("word2vec.model")
# model = Word2Vec(common_texts, size=100, window=5, min_count=1, workers=4)
# model.save("word2vec.model")
# model = Word2Vec.load("word2vec.model")
# model.train([all_desc_train], total_examples=1, epochs=1)

In [13]:
train_desc.shape, train_pic.shape, test_desc.shape, test_pic.shape

((10000, 512), (10000, 3048), (2000, 512), (2000, 3048))

In [14]:
train_desc.shape, train_pic.shape, test_desc.shape, test_pic.shape

((10000, 512), (10000, 3048), (2000, 512), (2000, 3048))

In [16]:
def get_prediction(vecs,pics):
    dists = pairwise_distances(vecs,pics,metric='cosine')
    return dists.T.argsort(1)
def map_20(ranks):
    return np.mean([(20-rank)/20 if rank<20 else 0 for rank in ranks])
def evaluate(vectors,label_vectors):
    preds = get_prediction(vectors,label_vectors)
    ranks = [np.argwhere(vec==i)[0][0] for i,vec in enumerate(preds)]
    return np.mean(ranks),map_20(ranks)
def get_top_20(descr_id):
    return preds[descr_id][:20]
def save_submission():
    data = []
    for i in range(2000):
        data.append(['%d.txt' % (i,),' '.join('%d.jpg' % (pic_id,) for pic_id in get_top_20(i))])
    pd.DataFrame(data,columns=['Descritpion_ID','Top_20_Image_IDs']).to_csv('submission.csv',index=False)

In [17]:
from sklearn.linear_model import RidgeCV

In [23]:
kf = KFold(n_splits=5)
rcv = RidgeCV(alphas=np.linspace(1.3,8,20))
for train_index, test_index in kf.split(train_pic):
    rcv.fit(train_pic[train_index], train_desc[train_index])
    pred = rcv.predict(train_pic[test_index])
    output = evaluate(pred, train_desc[test_index])
    print(output)

(7.526, 0.7270999999999999)
(7.78, 0.7249249999999999)
(7.6995, 0.72385)
(7.1675, 0.73025)
(7.383, 0.719275)


In [ ]:
regr.fit(train_desc, train_pic)
prediction = regr.predict(test_desc)
preds = get_prediction(prediction, test_pic)
save_submission()

In [18]:
import xgboost as xgb

In [19]:
bst = xgb.XGBRegressor()

In [20]:
bst.fit(train_pic, train_desc)

[22:09:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)